In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("/kaggle/input/bmi-labels/labels_utf8.csv")

In [ ]:
df.head(5)

In [ ]:
df.drop(["Hair", "Eyes", "Race", "Sex Offender", "Offense"], axis = 1, inplace = True)

In [ ]:
df.head(5)

In [ ]:
print(df.dtypes)

In [ ]:
df.isnull().values.any()

In [ ]:
import os

In [ ]:
!pip install dlib

In [ ]:
import dlib
import os
import numpy as np
from PIL import Image

In [ ]:
# Initialize dlib's face detector (HOG-based)
detector = dlib.get_frontal_face_detector()

# Path to the directory containing images to process
path = "/kaggle/input/sideimage/Dataset/front/front"

In [ ]:
# Arrays to store labels and cropped images
X = []
Y = []

In [ ]:
# Iterate through each file in the directory
for filename in os.listdir(path):
    if filename.endswith('.jpg'):
        # Construct absolute file path
        file_path = os.path.join(path, filename)
        
        # Load the image
        image = dlib.load_rgb_image(file_path)
        
        # Detect faces in the image
        detected_faces = detector(image, 1)
        
        # If at least one face is detected
        if len(detected_faces) > 0:
            # For each face detected
            for index, face_rect in enumerate(detected_faces):
                # Get the bounding box of the face
                x, y, w, h = (face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height())

                # Crop the face (convert dlib's image to a PIL Image first)
                face_image = Image.fromarray(image).crop((x, y, x+w, y+h))
                
                # Convert the cropped face image to a NumPy array and append to X
                face_array = np.array(face_image)
                X.append(face_array)
                
                # Store the label in array Y (strip the .jpg extension)
                label = filename.split('.')[0]
                Y.append(label)
        else:
            print(f"No face detected in {filename}")

In [ ]:
print(len(X))
print(len(Y))

In [ ]:
import numpy as np
import os

In [ ]:
pip install facenet-pytorch

In [ ]:
import torch
from facenet_pytorch import InceptionResnetV1
import torchvision.transforms.functional as F
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

In [ ]:
pip install torchvision

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def compute_face_embeddings(batch):
    # Define the transformation pipeline
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image
        transforms.ToTensor(),  # Convert the image to a tensor
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor
    ])

    # Convert numpy arrays to PIL Images if necessary and apply transformations
    transformed_batch = []
    for img in batch:
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)
        transformed_img = transform(img)
        transformed_batch.append(transformed_img)

    # Stack the transformed images into a batch
    batch_tensors = torch.stack(transformed_batch).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Compute embeddings
    with torch.no_grad():
        embeddings = resnet(batch_tensors).cpu().numpy()
    
    return embeddings


In [ ]:
directory = "/kaggle/input/frontcrop/frontcropped"

In [ ]:
#Not using for now
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load the MobileNet model, excluding the top (fully connected) layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model to prevent them from being updated during the first training phase
for layer in base_model.layers:
    layer.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer with 256 units and a relu activation
x = Dense(256, activation='relu')(x)

# Create the model that will output the 256-dimensional embeddings
modelu = Model(inputs=base_model.input, outputs=x)

modelu.summary()

In [ ]:
# Function to load images in batches
def load_images_in_batches(directory, batch_size=1000):
    filenames = os.listdir(directory)
    for i in range(0, len(filenames), batch_size):
        batch_files = filenames[i:i+batch_size]
        batch_images = []
        for file in batch_files:
            img_path = os.path.join(directory, file)
            img = image.load_img(img_path, target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)
            batch_images.append(img)
        yield np.vstack(batch_images), batch_files

In [ ]:
# Prepare to save embeddings
embeddings = []
labels = []

In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


In [ ]:
# Process images in batches
for images_batch, filenames in load_images_in_batches(directory):
    # Generate embeddings
    batch_embeddings = modelu.predict(images_batch)
    embeddings.extend(batch_embeddings)
    labels.extend([filename.split('.')[0] for filename in filenames])

In [ ]:
print(len(embeddings))
print(len(labels))

In [ ]:
embeddings[3]

In [ ]:
csv_file_path = '/kaggle/working/embeddings.csv'

In [ ]:
# Convert embeddings and labels into a DataFrame
df = pd.DataFrame(embeddings)
df['label'] = labels

In [ ]:
df.to_csv(csv_file_path, index=False)

In [2]:
import pandas as pd

In [4]:
pit = pd.read_csv("/kaggle/input/frontembeddings-csv/embeddings.csv")
pot = pd.read_csv("/kaggle/input/bmi-labels/labels_utf8.csv")

In [5]:
pit.head(19)

,label,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,M11813,0.111665,0.000000,0.000000,0.087155,0.000000,0.283555,0.000000,0.000000,0.0,...,0.000000,0.000000,0.945229,0.000000,1.393281,2.282689,0.0,0.973255,0.000000,0.000000
1,M53326,0.000000,0.000000,0.000000,0.267790,0.000000,0.466448,0.000000,0.183102,0.0,...,0.501978,0.000000,2.252731,0.000000,1.852022,3.060644,0.0,0.076265,0.000000,0.000000
2,Y22950,1.067019,0.000000,0.000000,0.354018,0.000000,0.774819,0.000000,0.000000,0.0,...,0.000000,0.000000,0.651599,0.000000,1.578468,1.465300,0.0,0.529442,0.000000,0.417858
3,S09796,0.000000,0.000000,0.000000,0.000000,0.000000,1.491947,0.000000,0.575940,0.0,...,0.000000,0.000000,0.378559,0.000000,1.444114,2.393884,0.0,1.355780,0.000000,0.000000
4,Y29851,0.000000,0.000000,0.000000,0.000000,0.000000,1.106095,0.000000,0.676816,0.0,...,0.608241,0.000000,0.362727,0.580366,0.235768,1.732123,0.0,0.844815,0.000000,0.000000
5,Y13293,0.000000,0.000000,0.000000,0.138999,0.613604,1.378825,0.000000,1.304752,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.650397,0.0,0.383372,0.000000,0.153848
6,M11887,0.000000,0.000000,0.000000,0.000000,0.000000,0.310248,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,2.543493,2.195034,0.0,1.370752,0.000000,0.654439
7,R59503,0.698034,0.000000,0.000000,0.000000,0.000000,0.359855,0.000000,0.164804,0.0,...,1.012527,0.482344,0.170199,0.000000,0.085531,3.104510,0.0,1.542924,0.000000,0.000000
8,B58525,0.000000,0.000000,0.000000,0.000000,0.236257,0.000000,0.000000,0.133390,0.0,...,0.000000,0.000000,0.000000,0.000000,1.346857,3.139775,0.0,0.596800,0.000000,0.431893
9,N71687,0.000000,0.000000,0.000000,0.000000,0.341415,1.964891,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,1.692624,2.699712,0.0,0.622131,0.000000,0.345724


In [6]:
pot.head()

,ID,Sex,Height,Weight,Hair,Eyes,Race,Sex Offender,Offense
0,A00147,Male,5 ft. 07 in.,185 lbs.,Brown,Blue,White,True,"AGGR KIDNAPPING-NO RANSOM, ATTEMPT MURDER/INTE..."
1,A00198,Male,6 ft. 02 in.,190 lbs.,Red or Auburn,Brown,White,False,"CRIM DMG TO PROP $300-10K, ATTEMPT AGG BATTERY..."
2,A00360,Male,5 ft. 09 in.,167 lbs.,Gray or Partially Gray,Green,White,False,"ARMED ROBBERY, OBSTRUCTING JUSTICE, CARRY/POSS..."
3,A00367,Male,6 ft. 00 in.,245 lbs.,Black,Brown,Black,False,"THEFT CONTROL INTENT 10K, RET THEFT/DISP MERCH..."
4,A01054,Male,5 ft. 07 in.,166 lbs.,Salt and Pepper,Brown,Black,True,"AGG CRIM SEX ASSAULT/WEAPON, ARMED ROBBERY, AG..."


In [25]:
# Function to convert height from 'ft. and in.' to 'inches'
def height_to_inches(height):
    # Split the string by space and remove the 'ft.' and 'in.' parts
    parts = height.lower().replace('ft.', '').replace('in.', '').split()
    feet = int(parts[0]) if parts[0].isdigit() else 0
    inches = int(parts[1]) if parts[1].isdigit() else 0
    return feet * 12 + inches

# Function to convert weight to pounds
def weight_to_pounds(weight):
    # Remove all characters except the numeric and the negative sign
    weight_part = ''.join(filter(str.isdigit, weight.replace('lbs', '')))
    return int(weight_part) if weight_part.isdigit() else 0

In [26]:
# Convert 'Height' to total inches
pot['Height_in_inches'] = pot['Height'].apply(height_to_inches)

# Convert 'Weight' to pounds
pot['Weight_in_pounds'] = pot['Weight'].apply(weight_to_pounds)

In [27]:
# Calculate BMI
pot['BMI'] = 703 * pot['Weight_in_pounds'] / pot['Height_in_inches']**2

In [28]:
# Display the updated DataFrame with the BMI column
print(pot.head())

       ID   Sex        Height    Weight                    Hair   Eyes   Race  \
0  A00147  Male  5 ft. 07 in.  185 lbs.                   Brown   Blue  White   
1  A00198  Male  6 ft. 02 in.  190 lbs.           Red or Auburn  Brown  White   
2  A00360  Male  5 ft. 09 in.  167 lbs.  Gray or Partially Gray  Green  White   
3  A00367  Male  6 ft. 00 in.  245 lbs.                   Black  Brown  Black   
4  A01054  Male  5 ft. 07 in.  166 lbs.         Salt and Pepper  Brown  Black   

   Sex Offender                                            Offense  \
0          True  AGGR KIDNAPPING-NO RANSOM, ATTEMPT MURDER/INTE...   
1         False  CRIM DMG TO PROP $300-10K, ATTEMPT AGG BATTERY...   
2         False  ARMED ROBBERY, OBSTRUCTING JUSTICE, CARRY/POSS...   
3         False  THEFT CONTROL INTENT 10K, RET THEFT/DISP MERCH...   
4          True  AGG CRIM SEX ASSAULT/WEAPON, ARMED ROBBERY, AG...   

   Height_in_inches  Weight_in_pounds        BMI  
0                67               185  28

In [32]:
# Now you can drop the temporary columns if you want
pot.drop(['Offense', 'Hair', 'Eyes', 'Race', 'Sex Offender'], axis=1, inplace=True)

In [37]:
# Display the updated DataFrame with the BMI column
pot.head()

,ID,Sex,Height,Weight,BMI
0,A00147,Male,5 ft. 07 in.,185 lbs.,28.971931
1,A00198,Male,6 ft. 02 in.,190 lbs.,24.391892
2,A00360,Male,5 ft. 09 in.,167 lbs.,24.658895
3,A00367,Male,6 ft. 00 in.,245 lbs.,33.224344
4,A01054,Male,5 ft. 07 in.,166 lbs.,25.996436


In [38]:
pot.drop(['Sex', 'Height', 'Weight'], axis=1, inplace=True)

In [40]:
pot.head()

,ID,BMI
0,A00147,28.971931
1,A00198,24.391892
2,A00360,24.658895
3,A00367,33.224344
4,A01054,25.996436


In [39]:
pit.head()

,label,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,M11813,0.111665,0.0,0.0,0.087155,0.0,0.283555,0.0,0.000000,0.0,...,0.000000,0.0,0.945229,0.000000,1.393281,2.282689,0.0,0.973255,0.0,0.000000
1,M53326,0.000000,0.0,0.0,0.267790,0.0,0.466448,0.0,0.183102,0.0,...,0.501978,0.0,2.252731,0.000000,1.852022,3.060644,0.0,0.076265,0.0,0.000000
2,Y22950,1.067019,0.0,0.0,0.354018,0.0,0.774819,0.0,0.000000,0.0,...,0.000000,0.0,0.651599,0.000000,1.578468,1.465300,0.0,0.529442,0.0,0.417858
3,S09796,0.000000,0.0,0.0,0.000000,0.0,1.491947,0.0,0.575940,0.0,...,0.000000,0.0,0.378559,0.000000,1.444114,2.393884,0.0,1.355780,0.0,0.000000
4,Y29851,0.000000,0.0,0.0,0.000000,0.0,1.106095,0.0,0.676816,0.0,...,0.608241,0.0,0.362727,0.580366,0.235768,1.732123,0.0,0.844815,0.0,0.000000


In [47]:
pot['ID'] = pot['ID'].astype(str)
pit['label'] = pit['label'].astype(str)

In [48]:
merged_df = pit.merge(pot, left_on='label', right_on='ID', how='left')
# The resulting `merged_df` will have all columns from `pit` plus the 'BMI' column from `pot`
# If the 'ID' column from `pot` is not needed, you can drop it
merged_df.drop('ID', axis=1, inplace=True)

In [49]:
merged_df.head()

,label,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,BMI
0,M11813,0.111665,0.0,0.0,0.087155,0.0,0.283555,0.0,0.000000,0.0,...,0.0,0.945229,0.000000,1.393281,2.282689,0.0,0.973255,0.0,0.000000,27.435721
1,M53326,0.000000,0.0,0.0,0.267790,0.0,0.466448,0.0,0.183102,0.0,...,0.0,2.252731,0.000000,1.852022,3.060644,0.0,0.076265,0.0,0.000000,27.152471
2,Y22950,1.067019,0.0,0.0,0.354018,0.0,0.774819,0.0,0.000000,0.0,...,0.0,0.651599,0.000000,1.578468,1.465300,0.0,0.529442,0.0,0.417858,29.531611
3,S09796,0.000000,0.0,0.0,0.000000,0.0,1.491947,0.0,0.575940,0.0,...,0.0,0.378559,0.000000,1.444114,2.393884,0.0,1.355780,0.0,0.000000,27.259184
4,Y29851,0.000000,0.0,0.0,0.000000,0.0,1.106095,0.0,0.676816,0.0,...,0.0,0.362727,0.580366,0.235768,1.732123,0.0,0.844815,0.0,0.000000,26.578450


In [50]:
merged_df.to_csv('/kaggle/working/merged.csv', index=False)  # Replace with your desired file path

In [51]:
data = pd.read_csv("/kaggle/input/embed-front/merged.csv")

In [52]:
data.head()

,label,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,BMI
0,M11813,0.111665,0.0,0.0,0.087155,0.0,0.283555,0.0,0.000000,0.0,...,0.0,0.945229,0.000000,1.393281,2.282689,0.0,0.973255,0.0,0.000000,27.435721
1,M53326,0.000000,0.0,0.0,0.267790,0.0,0.466448,0.0,0.183102,0.0,...,0.0,2.252731,0.000000,1.852022,3.060644,0.0,0.076265,0.0,0.000000,27.152471
2,Y22950,1.067019,0.0,0.0,0.354018,0.0,0.774819,0.0,0.000000,0.0,...,0.0,0.651599,0.000000,1.578468,1.465300,0.0,0.529442,0.0,0.417858,29.531611
3,S09796,0.000000,0.0,0.0,0.000000,0.0,1.491947,0.0,0.575940,0.0,...,0.0,0.378559,0.000000,1.444114,2.393884,0.0,1.355780,0.0,0.000000,27.259184
4,Y29851,0.000000,0.0,0.0,0.000000,0.0,1.106095,0.0,0.676816,0.0,...,0.0,0.362727,0.580366,0.235768,1.732123,0.0,0.844815,0.0,0.000000,26.578450


Training the dataset

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Load the data
data = pd.read_csv('/kaggle/input/embed-front/merged.csv')

In [4]:
# Split the data into features and labels
X = data.iloc[:, 1:-1]  # all columns from '0' to '255' (face embeddings)
y = data['BMI']  # the 'BMI' column

In [5]:
print(X.isnull().sum().sum())
print(y.isnull().sum())

0
340


In [6]:
# Option 1: Remove rows with NaN BMI
data_cleaned = data.dropna(subset=['BMI'])

In [7]:
print("NaNs in BMI after:", data_cleaned['BMI'].isnull().sum())

NaNs in BMI after: 0


In [8]:
# Split the data into features and labels after cleaning or imputation
X = data_cleaned.iloc[:, 1:-1]  # all columns from '0' to '255' (face embeddings)
y = data_cleaned['BMI']  # the 'BMI' column after cleaning or imputation

In [9]:
# Normalize the feature data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

Simple Neural Network Model for trails

In [11]:
# Define a simple neural network model
model_train = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output node for regression
])

In [12]:
# Compile the model
model_train.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# Train the model
history = model_train.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
1384/1384 [==============================] - 6s 4ms/step - loss: 36.8960 - val_loss: 28.5618
Epoch 2/50
1384/1384 [==============================] - 5s 4ms/step - loss: 26.3090 - val_loss: 28.7176
Epoch 3/50
1384/1384 [==============================] - 5s 4ms/step - loss: 24.6261 - val_loss: 27.5185
Epoch 4/50
1384/1384 [==============================] - 5s 4ms/step - loss: 23.5227 - val_loss: 27.3655
Epoch 5/50
1384/1384 [==============================] - 5s 4ms/step - loss: 22.4787 - val_loss: 27.1333
Epoch 6/50
1384/1384 [==============================] - 5s 4ms/step - loss: 21.7207 - val_loss: 27.1596
Epoch 7/50
1384/1384 [==============================] - 6s 4ms/step - loss: 20.6516 - val_loss: 28.2002
Epoch 8/50
1384/1384 [==============================] - 5s 4ms/step - loss: 19.9531 - val_loss: 27.8526
Epoch 9/50
1384/1384 [==============================] - 5s 4ms/step - loss: 19.0719 - val_loss: 28.1497
Epoch 10/50
1384/1384 [==============================] - 5s 4ms/

Loss is decreasing but val_loss is increasing, it indiacates, overfitting. 

In [14]:
# Evaluate the model on the test data
loss = model_train.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

433/433 [==============================] - 1s 2ms/step - loss: 37.9350
Test Loss: 37.93495178222656


In [15]:
# Predictions
y_pred = model_train.predict(X_test)

# Calculate accuracy or other regression metrics
# For regression, accuracy is not a meaningful metric. Instead, use RMSE, MSE, or MAE.
mse = keras.metrics.mean_squared_error(y_test, y_pred)
rmse = keras.backend.sqrt(mse)
mae = keras.metrics.mean_absolute_error(y_test, y_pred)

print(f'Test MSE: {mse.numpy()}')
print(f'Test RMSE: {rmse.numpy()}')
print(f'Test MAE: {mae.numpy()}')

433/433 [==============================] - 1s 2ms/step
Test MSE: [ 26.817083  90.09427   40.43979  ...  37.289013  41.3717   125.17625 ]
Test RMSE: [ 5.178521   9.4918     6.3592286 ...  6.106473   6.4320836 11.188219 ]
Test MAE: [ 3.984181   8.00723    4.673185  ...  5.1163516  5.449193  10.282669 ]


In [19]:
import numpy as np

# Ensure y_pred is flattened (since your model's output might have extra dimensions)
y_pred_flattened = y_pred.flatten()

# Calculate Pearson correlation coefficient
correlation_matrix = np.corrcoef(y_test, y_pred_flattened)
pearson_correlation = correlation_matrix[0, 1]

print(f'Pearson Correlation Coefficient: {pearson_correlation}')

Pearson Correlation Coefficient: 0.20170629538044893


In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [79]:
# Initialize the linear regression model
lr_model = LinearRegression()

# Fit the model on the training data
lr_model.fit(X_train, y_train)

# Predict on the test data
y_pred_lr = lr_model.predict(X_test)

# Calculate the MSE, RMSE, and MAE
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f'Linear Regression Test MSE: {mse_lr}')
print(f'Linear Regression Test RMSE: {rmse_lr}')
print(f'Linear Regression Test MAE: {mae_lr}')

Linear Regression Test MSE: 24.295604365655585
Linear Regression Test RMSE: 4.9290571477368355
Linear Regression Test MAE: 3.7660950877512005


In [80]:
from scipy.stats import pearsonr

In [82]:
correlation, _ = pearsonr(y_test, y_pred_lr)

In [83]:
print(f'Pearson Correlation Coefficient: {correlation}')

Pearson Correlation Coefficient: 0.3073585559253895


Random Forest Regressor for Regression


In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [23]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [24]:
# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f'Random Forest Test MSE: {mse_rf}')
print(f'Random Forest Test MAE: {mae_rf}')

# Calculate the Pearson correlation coefficient for the Random Forest model
pearson_corr_rf, _ = pearsonr(y_test, y_pred_rf)
print(f'Random Forest Pearson Correlation Coefficient: {pearson_corr_rf}')

Random Forest Test MSE: 24.786031962346957
Random Forest Test MAE: 3.829730074226885
Random Forest Pearson Correlation Coefficient: 0.2789699908036613


#  Using Vision Transformers

![](https://www.mdpi.com/diagnostics/diagnostics-13-02459/article_deploy/html/images/diagnostics-13-02459-g001.png)



In [1]:
pip install torch torchvision transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from torch.utils.data import Dataset, DataLoader

In [3]:
# Load BMI values
data_path = "/kaggle/input/embed-front/merged.csv"
bmi_data = pd.read_csv(data_path)

In [4]:
# Check for 'nan' or 'inf' values in BMI and drop those rows
if not bmi_data['BMI'].apply(np.isfinite).all():
    print("BMI data contains nan or inf. Dropping those rows.")
    bmi_data = bmi_data.dropna(subset=['BMI'])
    bmi_data = bmi_data[np.isfinite(bmi_data['BMI'])]

BMI data contains nan or inf. Dropping those rows.


In [5]:
# Check for any 'nan' or 'inf' values in BMI and drop those rows
if not np.isfinite(bmi_data['BMI']).all():
    print("Data contains nan or inf. Dropping those rows.")
    bmi_data = bmi_data[np.isfinite(bmi_data['BMI'])]

In [6]:
from torchvision.transforms import Compose, Resize, Normalize
from torch.utils.data import Dataset, DataLoader

# Define transformations
transform = Compose([
    Resize((224, 224), antialias=True),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
from torchvision.transforms.functional import convert_image_dtype

In [8]:
class BMIDataset(Dataset):
    def __init__(self, img_dir, bmi_data, transform=None):
        self.img_dir = img_dir
        # Assuming the 'ID' column in bmi_data DataFrame contains the image labels without the .jpg extension
        self.bmi_data = bmi_data.set_index('label')  # Setting 'label' as index for easier lookup
        self.transform = transform

    def __len__(self):
        return len(self.bmi_data)

    def __getitem__(self, idx):
        img_label = self.bmi_data.index[idx]
        img_path = f"{self.img_dir}/{img_label}.jpg"
        image = read_image(img_path)  # This returns a tensor in torch.uint8
        image = convert_image_dtype(image, dtype=torch.float)  # Convert to float
        bmi = self.bmi_data.iloc[idx]['BMI']
        if self.transform:
            image = self.transform(image)
        return image, bmi

In [9]:
# Create the dataset and dataloader
img_dir = "/kaggle/input/frontcrop/frontcropped"

In [10]:
from sklearn.model_selection import train_test_split

# Assuming bmi_data is your DataFrame
train_data, test_data = train_test_split(bmi_data, test_size=0.2, random_state=42)  # 80% training, 20% testing

# Optionally split train_data further to create a validation set
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # Results in 60% train, 20% validation, 20% test

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
train_dataset = BMIDataset(img_dir, train_data, transform=transform)
val_dataset = BMIDataset(img_dir, val_data, transform=transform)  # If using a validation set
test_dataset = BMIDataset(img_dir, test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # If using a validation set
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [16]:
import torch

In [17]:
# Check images for anomalies
for i, (imgs, bmis) in enumerate(train_loader):
    if not torch.isfinite(imgs).all():
        print(f"Images at batch index {i} contain nan or inf.")
    if not np.isfinite(bmis.numpy()).all():  # Convert entire tensor to numpy and then check with np.isfinite
        print(f"BMI values at batch index {i} contain nan or inf.")
        
# Check images for anomalies
for i, (imgs, bmis) in enumerate(val_loader):
    if not torch.isfinite(imgs).all():
        print(f"Images at batch index {i} contain nan or inf.")
    if not np.isfinite(bmis.numpy()).all():  # Convert entire tensor to numpy and then check with np.isfinite
        print(f"BMI values at batch index {i} contain nan or inf.")
        

# Check images for anomalies
for i, (imgs, bmis) in enumerate(test_loader):
    if not torch.isfinite(imgs).all():
        print(f"Images at batch index {i} contain nan or inf.")
    if not np.isfinite(bmis.numpy()).all():  # Convert entire tensor to numpy and then check with np.isfinite
        print(f"BMI values at batch index {i} contain nan or inf.")

In [18]:
import torch

In [22]:
# Correct usage of DataLoader in a for loop
for i, (img, bmi) in enumerate(train_loader):
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    if not np.isfinite(bmi.numpy()).all():  # Use .all() for batched data
        print(f"BMI value at batch index {i} is nan or inf.")
        
for i, (img, bmi) in enumerate(val_loader):
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    if not np.isfinite(bmi.numpy()).all():  # Use .all() for batched data
        print(f"BMI value at batch index {i} is nan or inf.")

for i, (img, bmi) in enumerate(test_loader):
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    if not np.isfinite(bmi.numpy()).all():  # Use .all() for batched data
        print(f"BMI value at batch index {i} is nan or inf.")

In [24]:
import numpy as np

# Check images for anomalies
for i in range(len(train_dataset)):
    img, bmi = train_dataset[i]
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    # Assuming bmi is a scalar float or numpy.float64 at this point, not a tensor
    if not np.isfinite(bmi):  # Correct use of np.isfinite for scalar values
        print(f"BMI value at index {i} is nan or inf.")
        
# Check images for anomalies
for i in range(len(val_dataset)):
    img, bmi = val_dataset[i]
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    # Assuming bmi is a scalar float or numpy.float64 at this point, not a tensor
    if not np.isfinite(bmi):  # Correct use of np.isfinite for scalar values
        print(f"BMI value at index {i} is nan or inf.")

# Check images for anomalies
for i in range(len(test_dataset)):
    img, bmi = test_dataset[i]
    if not torch.isfinite(img).all():
        print(f"Image at index {i} contains nan or inf.")
    # Assuming bmi is a scalar float or numpy.float64 at this point, not a tensor
    if not np.isfinite(bmi):  # Correct use of np.isfinite for scalar values
        print(f"BMI value at index {i} is nan or inf.")



In [25]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torch import nn

In [26]:
# Load pre-trained ViT model
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Modify the model for regression
model.classifier = nn.Linear(model.config.hidden_size, 1) # Assuming the output is a single continuous value for BMI

In [28]:
import torch
from torch.optim import Adam
from torch.nn import MSELoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-5)
loss_fn = MSELoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, bmis in train_loader:
        images = images.to(device)
        bmis = bmis.to(device).float().unsqueeze(1)  # Ensure BMI is the correct shape

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = loss_fn(outputs, bmis)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_loss = 0
    with torch.no_grad():
        val_loss = 0
        for images, bmis in val_loader:
            images = images.to(device)
            bmis = bmis.to(device).float().unsqueeze(1)
            outputs = model(images).logits
            loss = loss_fn(outputs, bmis)
            # Your validation process here
            val_loss += loss.item()
        val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss}")

Epoch 1, Training Loss: 445.8351135253906, Validation Loss: 511.3405130223369
Epoch 2, Training Loss: 318.3039245605469, Validation Loss: 374.5161909495435
Epoch 3, Training Loss: 202.1973114013672, Validation Loss: 247.99044704657504
Epoch 4, Training Loss: 111.11869049072266, Validation Loss: 145.98374954822708
Epoch 5, Training Loss: 51.985435485839844, Validation Loss: 76.04647754026065
Epoch 6, Training Loss: 25.26141357421875, Validation Loss: 39.74374608795307
Epoch 7, Training Loss: 20.916349411010742, Validation Loss: 28.744338101657927
Epoch 8, Training Loss: 22.36387062072754, Validation Loss: 27.615343087264627
Epoch 9, Training Loss: 20.633249282836914, Validation Loss: 26.892218466443772
Epoch 10, Training Loss: 18.58112335205078, Validation Loss: 22.522558715547206


In [30]:
model.eval()  # Set the model to evaluation mode
test_loss = 0

with torch.no_grad():
    for images, bmis in test_loader:
        images = images.to(device)  # Move images to the same device as the model
        bmis = bmis.to(device).float().unsqueeze(1)  # Ensure BMIs are on the same device and match expected shape

        outputs = model(images).logits  # Assuming your model returns logits directly
        loss = loss_fn(outputs, bmis)
        test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss}")


Test Loss: 21.97793846372644


# Accuracy

In [32]:
model.eval()  # Set the model to evaluation mode
predictions = []  # Initialize an empty list to store predictions
actuals = []  # Initialize an empty list to store actual BMI values

with torch.no_grad():
    for images, bmis in test_loader:
        images = images.to(device)
        bmis = bmis.to(device).float().unsqueeze(1)
        
        # Get model predictions
        output = model(images).logits.squeeze(1)  # Adjust depending on your model's output shape
        predictions.extend(output.cpu().numpy())  # Append predictions to the list
        actuals.extend(bmis.cpu().numpy().squeeze(1))  # Append actual BMI values to the list

# Now you have populated 'predictions' and 'actuals', you can convert them to numpy arrays
predictions = np.array(predictions)
actuals = np.array(actuals)

# Then calculate the metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(actuals, predictions)
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(actuals, predictions)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")



Mean Absolute Error (MAE): 3.4764
Mean Squared Error (MSE): 21.9643
Root Mean Squared Error (RMSE): 4.6866
R-squared (R²): 0.1807


In [33]:
from scipy.stats import pearsonr

In [34]:
# Calculate Pearson correlation coefficient
pearson_corr, _ = pearsonr(actuals, predictions)
print(f"Pearson Correlation Coefficient: {pearson_corr:.4f}")


Pearson Correlation Coefficient: 0.4525
